# Data Science 100 Final Project Report

In [1]:
#Loading the relevant R package:
library(tidyverse)
library(tidymodels)
library(repr)
library(themis)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

### Method and Results

In [ ]:
# 1. Cleaning and wrangling players.csv 
players_cleaned <- players|>
                   filter(!is.na(Age))|>
                   mutate(subscribe = as.factor(subscribe))

players_cleaned

In [ ]:
# 2. Summary statistics (mean values for each numerical variables)
players_average <- players_cleaned |>
                   summarize(average_played_hours = mean(played_hours), average_age = mean(Age))
players_average

In [ ]:
# 3. splitting dataframe into training, testing datasets
set.seed(1)
players_split <- initial_split(players_cleaned, prop = 0.75, strata = subscribe)

players_training <- training(players_split)
players_testing <- testing(players_split)

spec<- nearest_neighbor(weight_func="rectangular", neighbors=tune())|>
set_engine("kknn")|>
set_mode("classification")

recipe<- recipe(subscribe~Age+played_hours, data=players_training)|>
step_scale(all_predictors())|>
step_center(all_predictors()) |>
step_upsample(subscribe, over_ratio = 1)

k<- tibble(neighbors=seq(from=1, to=100, by=1))
vfold<- vfold_cv(players_training, v=5, strata=subscribe)

workflow<- workflow()|>
add_recipe(recipe)|>
add_model(spec)|>
tune_grid(resamples=vfold, grid=k)|>
collect_metrics()
accuracy<-workflow|>
filter(.metric=="accuracy")
accuracy
best<- filter(accuracy, mean==max(mean)) |>
filter(neighbors = min(neighbors)) |>

best

In [ ]:
ggplot_accuracy<- ggplot(accuracy, aes(x=neighbors, y=mean))+
geom_point()+
geom_line()
ggplot_accuracy

In [ ]:
test_try_spec<- nearest_neighbor(weight_func = "rectangular", neighbors=19)|>
set_engine("kknn")|>
set_mode("classification")

test_fit<- workflow() |>
  add_recipe(recipe) |>
  add_model(test_try_spec) |>
  fit(data = players_training)

test_predictions <- predict(test_fit, players_testing) |>
  bind_cols(players_testing)

test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

test_predictions |>
    precision(truth = subscribe, estimate = .pred_class, event_level="first")

test_predictions |>
    recall(truth = subscribe, estimate = .pred_class, event_level="first")
test_predictions

confusion <- test_predictions |>
             conf_mat(truth = subscribe, estimate = .pred_class)
confusion

In [ ]:
# 4. Exploratory visualizations
# a) finding relationships for the Age predictor:
options(repr.plot.width = 8, repr.plot.height = 8)
Age_boxplot <- ggplot(players_training, aes(x = subscribe, y = Age, fill = subscribe)) +
               geom_boxplot() +
               labs(x= "Subscription Status", y= "Age of Player", fill = "Subscription\nStatus", title = "Age vs. Subscription Status")+
               theme(text = element_text(size = 15))
Age_boxplot

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
Age_histogram <- ggplot(players_training, aes(x = Age, fill = subscribe)) +
                 geom_histogram() +
                 facet_grid(cols = vars(subscribe))+
                 labs(x = "Age Distribution", y = "Subscription Count", fill = "Subscription\nStatus", title = "Age Distribution by Subscription Status")+
                 theme(text = element_text(size = 15))
Age_histogram

In [ ]:
# b) finding relationships for the played_hours predictor:
options(repr.plot.width = 10, repr.plot.height = 8)
played_hours_boxplot <- ggplot(players_training, aes(x = subscribe, y = played_hours, fill = subscribe)) +
                        geom_boxplot() +
                        scale_y_log10(labels=label_comma())+
                        labs(x = "Subscription Status", y = "Game Played Hours", fill = "Subscription\nStatus", title = "played hours vs. Subscription Status")+
                        theme(text = element_text(size = 15))
played_hours_boxplot